Web Scraping using Python (and Beautiful Soup)

Web scraping is a term used to describe the use of a program or algorithm to extract and process large amounts of data from the web. Whether you are a data scientist, engineer, or anybody who analyzes large amounts of datasets, the ability to scrape data from the web is a useful skill to have. Let's say you find data from the web, and there is no direct way to download it, web scraping using Python is a skill you can use to extract the data into a useful form that can be imported.

In this tutorial, you will learn about the following:

Data extraction from the web using Python's Beautiful Soup module
Data manipulation and cleaning using Python's Pandas library
Data visualization using Python's Matplotlib library
The dataset used in this tutorial was taken from a 10K race that took place in Hillsboro, OR on June 2017. Specifically, you will analyze the performance of the 10K runners and answer questions such as:

What was the average finish time for the runners?
Did the runners' finish times follow a normal distribution?
Were there any performance differences between males and females of various age groups?
Web Scraping using Beautiful Soup

Using Jupyter Notebook, you should start by importing the necessary modules (pandas, numpy, matplotlib.pyplot, seaborn). If you don't have Jupyter Notebook installed, I recommend installing it using the Anaconda Python distribution which is available on the internet. To easily display the plots, make sure to include the line %matplotlib inline as shown below.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


To perform web scraping, you should also import the libraries shown below. The urllib.request module is used to open URLs. The Beautiful Soup package is used to extract data from html files. The Beautiful Soup library's name is bs4 which stands for Beautiful Soup, version 4.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [ ]:
url = "http://www.hubertiming.com/results/2017GPTR10K"
html = urlopen(url)


In [ ]:
soup = BeautifulSoup(html, 'lxml')
type(soup)


In [ ]:
bs4.BeautifulSoup


In [ ]:
# Get the title
title = soup.title
print(title)


In [ ]:
<title>2017 Intel Great Place to Run 10K \ Urban Clash Games Race Results</title>


In [ ]:
# Print out the text
text = soup.get_text()
#print(soup.text)


In [ ]:
soup.find_all('a')


In [ ]:
[<a class="btn btn-primary btn-lg" href="/results/2017GPTR" role="button">5K</a>,
 <a href="http://hubertiming.com">Huber Timing Home</a>,
 <a href="#individual">Individual Results</a>,
 <a href="#team">Team Results</a>,
 <a href="mailto:timing@hubertiming.com">timing@hubertiming.com</a>,
 <a href="#tabs-1" style="font-size: 18px">Results</a>,
 <a name="individual"></a>,
 <a name="team"></a>,
 <a href="http://www.hubertiming.com"><img height="65" src="/sites/all/themes/hubertiming/images/clockWithFinishSign_small.png" width="50"/>Huber Timing</a>,
 <a href="http://facebook.com/hubertiming"><img src="/results/FB-f-Logo__blue_50.png"/></a>]


In [ ]:
all_links = soup.find_all("a")
for link in all_links:
    print(link.get("href"))


In [ ]:
/results/2017GPTR
http://hubertiming.com/
#individual
#team
mailto:timing@hubertiming.com
#tabs-1
None
None
http://www.hubertiming.com/
http://facebook.com/hubertiming/


In [ ]:
# Print the first 10 rows for sanity check
rows = soup.find_all('tr')
print(rows[:10])


In [ ]:
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)


In [ ]:
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)


In [ ]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)


In [ ]:
df = pd.DataFrame(list_rows)
df.head(10)


In [ ]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)


In [ ]:
df1[0] = df1[0].str.strip('[')
df1.head(10)


In [ ]:
col_labels = soup.find_all('th')


In [ ]:
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
print(all_header)


In [ ]:
df2 = pd.DataFrame(all_header)
df2.head()


In [ ]:
df3 = df2[0].str.split(',', expand=True)
df3.head()


In [ ]:
frames = [df3, df1]

df4 = pd.concat(frames)
df4.head(10)


In [ ]:
df5 = df4.rename(columns=df4.iloc[0])
df5.head()


In [ ]:
df5.info()
df5.shape


In [ ]:
df6 = df5.dropna(axis=0, how='any')


In [ ]:
df7 = df6.drop(df6.index[0])
df7.head()


In [ ]:
df7.rename(columns={'[Place': 'Place'},inplace=True)
df7.rename(columns={' Team]': 'Team'},inplace=True)
df7.head()


In [ ]:
df7['Team'] = df7['Team'].str.strip(']')
df7.head()


In [ ]:
time_list = df7[' Chip Time'].tolist()

# You can use a for loop to convert 'Chip Time' to minutes

time_mins = []
for i in time_list:
    h, m, s = i.split(':')
    math = (int(h) * 3600 + int(m) * 60 + int(s))/60
    time_mins.append(math)
#print(time_mins)


In [ ]:
df7['Runner_mins'] = time_mins
df7.head()


In [ ]:
df7.describe(include=[np.number])


In [ ]:
df7.boxplot(column='Runner_mins')
plt.grid(True, axis='y')
plt.ylabel('Chip Time')
plt.xticks([1], ['Runners'])


In [ ]:
([<matplotlib.axis.XTick at 0x570dd106d8>],
 <a list of 1 Text xticklabel objects>)


In [ ]:
x = df7['Runner_mins']
ax = sns.distplot(x, hist=True, kde=True, rug=False, color='m', bins=25, hist_kws={'edgecolor':'black'})
plt.show()


In [ ]:
f_fuko = df7.loc[df7[' Gender']==' F']['Runner_mins']
m_fuko = df7.loc[df7[' Gender']==' M']['Runner_mins']
sns.distplot(f_fuko, hist=True, kde=True, rug=False, hist_kws={'edgecolor':'black'}, label='Female')
sns.distplot(m_fuko, hist=False, kde=True, rug=False, hist_kws={'edgecolor':'black'}, label='Male')
plt.legend()


In [ ]:
<matplotlib.legend.Legend at 0x570e301fd0>


In [ ]:
g_stats = df7.groupby(" Gender", as_index=True).describe()
print(g_stats)


In [ ]:
df7.boxplot(column='Runner_mins', by=' Gender')
plt.ylabel('Chip Time')
plt.suptitle("")
